In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install yfinance
!pip install newsapi-python
!pip install unidecode

     |████████████████████████████████| 5.5MB 10.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=004f93ed0f7016cc859200cc743c88292d6974f89b863850f11d7f56afd67891
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 245kB 8.8MB/s 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import GenericUnivariateSelect, chi2
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_curve
from sklearn.utils import resample
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from string import punctuation
from unidecode import unidecode
import re


from collections import Counter

#from lime import lime_text
#from lime.lime_text import LimeTextExplainer

#from category_encoders import *
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import time
import datetime
#import win32com.client as win32
from traitlets import traitlets

import random
import math

nltk.data.path.append('./nltk_data/')

import yfinance as yf

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df_news_before=pd.read_csv('/content/drive/MyDrive/IBovespa/valoreconomico/valoreconomico.csv',nrows=5000)

FileNotFoundError: ignored

In [ ]:
df_news_before['data'] = pd.to_datetime(df_news_before['data'])
df_news_before['flag'] = df_news_before['data'].apply(lambda x: 'depois_18' if x.hour > 18  else ('antes_das_9' if x.hour < 9 else 'intra'))
df_news_before['title'] = df_news_before['title'].apply(lambda x:x.split(sep='|')[0])
df_news_before['data_target'] = df_news_before.apply(lambda x:x.data + datetime.timedelta(days=1) if x.flag == 'depois_18' else x.data, axis=1)
df_news_before['data_target'] = df_news_before['data_target'].dt.strftime('%Y%m%d')
#del df_news['date_target']
#df_news['data_target'] = df_news['data_target'].apply(lambda x: datetime.datetime.strptime(x,'%y%m%d').date())
#somar um dia para as que estão depois das 18h

In [ ]:
df_news = df_news_before.query('flag != "intra"')[['title','data_target']]
df_news['data'] = df_news['data_target']
#del df_news['data_target']

In [ ]:
df_news_fechamento = df_news[df_news['flag']==1]
df_news_inter = df_news[df_news['flag']==0]

In [ ]:
df_group_fechamento = df_news.groupby(df_news['data'])['title'].sum().to_frame()
#df_group_int = df_news_inter.groupby(df_news_inter['data'].dt.date)['title'].sum().to_frame()

,title
data,
20110109,Vale Fertilizantes reverte prejuízo e lucra R$...
20110208,Kirin compra a Schincariol por R$ 4 bi Movimen...
20110209,CPFL investirá R$ 360 milhões em termelétricas...
20110212,Correção: Haddad deve ir a Recife para recorre...
20110308,CPI para investigar Transportes obtém assinatu...
20110309,"Lucro líquido da Marisa sobre 31,8% no trimest..."
20110311,Fisco é condenado por dano moral Agenda Pitada...
20110312,Lucro da SulAmérica cai 19% com aumento de ind...
20110408,Responsabilidade civil em contrato internacion...


# **FUNÇÃO CHICO**

In [ ]:
tickers = ['^BVSP']

start_date = "2000-01-01"
end_date = "2020-11-01"

data_bvsp = yf.download(tickers, start=start_date, end=end_date)

In [ ]:
df_group_fechamento['Result_Ibov'] = np.random.randint(0,2, (1,len(df_group_fechamento))).T
df_group_int['Result_Ibov'] = np.random.randint(0,2, (1,len(df_group_int))).T

In [ ]:
def clean_text(string):

  palavras_tokenize = word_tokenize(string, language='portuguese')

  filtered_words = [word for word in palavras_tokenize if word not in stopwords.words('portuguese')]
  return TreebankWordDetokenizer().detokenize(filtered_words)

In [ ]:
df_group_fechamento['title']=df_group_fechamento['title'].apply(lambda x:unidecode(x.lower()))
df_group_fechamento['title']=df_group_fechamento['title'].apply(lambda x:re.sub('[^a-zA-Z]', ' ', x))
df_group_fechamento['title']=df_group_fechamento['title'].apply(clean_text)

df_group_int['title']=df_group_int['title'].apply(lambda x:unidecode(x.lower()))
df_group_int['title']=df_group_int['title'].apply(lambda x:re.sub
                                                  ('[^a-zA-Z]', ' ', x))
df_group_int['title']=df_group_int['title'].apply(clean_text)

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))

X_train, X_test, y_train, y_test = train_test_split(df_group_fechamento['title'],
                                                    df_group_fechamento['Result_Ibov'],
                                                    test_size = 0.3,
                                                    random_state=42)

X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
  
model = MultinomialNB(alpha=4)

model.fit(X_train_vect, y_train)
y_pred_eval = model.predict_proba(X_test_vect)[:,1]

In [ ]:
dv = pd.DataFrame(zip(df_group_fechamento['title'], y_pred_eval), columns=['title', 'Result_Ibov'])
dv = dv.sort_values('Result_Ibov', ascending = False)
dv

,title,Result_Ibov
8,responsabilidade civil contrato internacional ...,0.458071
10,ritmo migracao populacional diminuiu regioes p...,0.453570
7,lucro sulamerica cai aumento indenizacoes fitc...,0.451365
12,esforco contra corrupcao deve ter apoio diz ma...,0.450132
1,kirin compra schincariol r bi movimento falime...,0.450048
11,corretora seguro willis novo presidente americ...,0.450001
9,dolar futuro fecha forte alta dilma anuncia no...,0.449344
2,cpfl investira r milhoes termeletricas pr mg r...,0.448943
6,fisco condenado dano moral agenda pitada risco...,0.447384
4,cpi investigar transportes obtem assinaturas p...,0.446519


array(['responsabilidade civil contrato internacional casal ministros renova politica pr brasileiro considera carro casa investimento empresa avalia comprar acoes usiminas bradesco condenado monitorar contas vale nao fala portugues apresentacoes balanco reorganizacao acionaria oi comeca esquentar drogasil sera parametro rentabilidade hiperinflacao afeta custos diz xstrata curta venda jatos movimenta us milhoes justica schincariol reavivam divergencias movimento falimentar bid pede brasil acabar tarifas importacao produtos colombianos chineses ameacam retaliar importacoes brasileiras filme basquete ajudam time warner mp eleva ipi cigarros cofins sobre importacoes crise agora atinge italia espanha sucessao fator correcoes commodities agricolas lider mcgraw hill decide futuro conglomerado centrais pressionarao congresso condicionantes desoneracao folha medida cambial vale poucos centavos eua precisam novo modelo negocios rim renova blackberry enfrentar iphone onda pre sal macae tera prime